In [ ]:
import os
import zipfile

local_zip = 'Data_T2.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/Data_T2'


train_leo_dir = os.path.join(base_dir, 'Leopardo2')
train_lince_dir = os.path.join(base_dir, 'Lince iberico')
train_ocelote_dir = os.path.join(base_dir, 'Ocelote')
train_guepardo_dir = os.path.join(base_dir, 'Guepardo')
train_serval_dir = os.path.join(base_dir, 'Serval')

train_leo_fnames = os.listdir( train_leo_dir )
train_lince_fnames = os.listdir( train_lince_dir )
train_ocelote_fnames = os.listdir(train_ocelote_dir)
train_guepardo_fnames = os.listdir(train_guepardo_dir)
train_serval_fnames = os.listdir(train_serval_dir)

print(train_leo_fnames[:10])
print(train_lince_fnames[:10])
print(train_ocelote_fnames[:10])
print(train_guepardo_fnames[:10])
print(train_serval_fnames[:10])

print('total training leo images :', len(os.listdir(train_leo_dir ) ))
print('total training lince images :', len(os.listdir( train_lince_dir ) ))
print('total training ocelote images :', len(os.listdir( train_ocelote_dir ) ))
print('total training guepardo images :', len(os.listdir( train_guepardo_dir ) ))
print('total training serval images :', len(os.listdir( train_serval_dir ) ))

['leopardo (124).jpg', 'leopardo (47).jpg', 'leopardo (110).jpg', 'leopardo (40).jpg', 'leopardo (156).jpg', 'leopardo (74).jpg', 'leopardo (44).jpg', 'leopardo (11).jpg', 'leopardo (16).jpg', 'leopardo (147).jpg']
['lince1 (50).jpg', 'lince1 (163).jpg', 'lince1 (19).jpg', 'lince1 (2).jpg', 'lince1 (14).jpg', 'lince1 (42).jpg', 'lince1 (72).jpg', 'lince1 (79).jpg', 'lince1 (108).jpg', 'lince1 (174).jpg']
['Ocelote (155).jpg', 'Ocelote (100).jpg', 'Ocelote (86).jpg', 'Ocelote (18).jpg', 'Ocelote (52).jpg', 'Ocelote (119).jpg', 'Ocelote (43).jpg', 'Ocelote (32).jpg', 'Ocelote (90).jpg', 'Ocelote (49).jpg']
['Guepardo (161).jpg', 'Guepardo (146).jpg', 'Guepardo (179).jpg', 'guepardo (129).jpg', 'guepardo (59).jpg', 'guepardo (67).jpg', 'Guepardo (192).jpg', 'guepardo (49).jpg', 'guepardo (41).jpg', 'Guepardo (154).jpg']
['serval (126).jpg', 'serval (13).jpg', 'Serval (161).jpg', 'Serval (84).jpg', 'serval (45).jpg', 'serval (139).jpg', 'serval (50).jpg', 'Serval (182).jpg', 'serval (62).j

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
! pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 5.3MB/s 


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#todas las imagenes sera resscaldad a 1/255
train_datagen = ImageDataGenerator( rescale = 1.0/255. ,validation_split=0.15)
# --------------------
# Imágenes de entrenamiento en lotes de 20 usando el generador train_datagen
# --------------------
train_generator = train_datagen.flow_from_directory(base_dir,
                                                    batch_size=80,
                                                    class_mode='categorical',
                                                    target_size=(150,150),
                                                    subset='training')     

# --------------------
# imagenes de validadcion en lotes de 5 usando el generador test_datagen generator
# --------------------

test_generator = train_datagen.flow_from_directory(base_dir,
                                                    batch_size=5,
                                                    class_mode='categorical',
                                                    target_size=(150, 150),
                                                    subset='validation') 

Found 816 images belonging to 5 classes.
Found 142 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.optimizers import RMSprop
def build_model(hp):
  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 32, 256, step=32)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(
        filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D()(x)
    else:
      x = tf.keras.layers.AvgPool2D()(x)
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  initializer=tf.keras.initializers.RandomNormal(mean=0.3, stddev=1.)
  #imputs2=tf.keras.layers.Dense(16,kernel_initializer=initializer,activation='relu')(y)
  x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 16, 1024, step=16, default=50),kernel_initializer=initializer,
      activation='relu')(x)
  x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)   
  #imputs2=tf.keras.layers.Dense(16,kernel_initializer=initializer,activation='relu')(x)
  outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
    optimizer=tf.keras.optimizers.RMSprop(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='categorical_crossentropy', 
    metrics=['accuracy'])
  return model

In [ ]:
import kerastuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

In [ ]:
#import tensorflow_datasets as tfds

#data = tfds.load('cifar10')
#train_ds, test_ds = data['train'], data['test']

#def standardize_record(record):
#  return tf.cast(record['image'], tf.float32) / 255., record['label']

#train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
#test_ds = test_ds.map(standardize_record).cache().batch(64)
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_generator,
             validation_data=test_generator,
             epochs=30,
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
             #[tf.keras.callbacks.EarlyStopping(patience=1)])

Trial 180 Complete [00h 01m 02s]
val_accuracy: 0.19718310236930847

Best val_accuracy So Far: 0.3239436745643616
Total elapsed time: 02h 12m 18s
INFO:tensorflow:Oracle triggered exit


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
#tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ValueError: ignored

In [ ]:
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 64)      256       
_________________________________________________________________
re_lu (ReLU)                 (None, 150, 150, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 64)      256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 150, 150, 64)      0     

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_hyperparameters